# TEST OF PETSC AND SLEPC

Here we experiement with PETSc and SLEPc libraries and their ports to Python. We are interested in solving linear systems $Ax = b$ and for finding eigenvalues and eigenvectors.

## ex2 in PETSc Demo
Solve a Linear System in Parallel with KSP. File can be found at https://bitbucket.org/petsc/petsc4py/src/master/demo/petsc-examples/ksp/ex2.py

In [ ]:
'''
Ex2 from PETSc example files implemented for PETSc4py.
https://www.mcs.anl.gov/petsc/petsc-current/src/ksp/ksp/examples/tutorials/ex2.c.html
By: Miguel Arriaga


Solves a linear system in parallel with KSP.
Input parameters include:
    -view_exact_sol   : write exact solution vector to stdout
    -m <mesh_x>       : number of mesh points in x-direction
    -n <mesh_n>       : number of mesh points in y-direction


Concepts: KSP^basic parallel example
Concepts: KSP^Laplacian, 2d
Concepts: Laplacian, 2d
Processors: n

Vec            x,b,u;    # approx solution, RHS, exact solution 
Mat            A;        # linear system matrix 
KSP            ksp;      # linear solver context 
PetscReal      norm;     # norm of solution error
'''
import sys
import petsc4py
petsc4py.init(sys.argv)
from petsc4py import PETSc

import numpy as np

In [ ]:
comm = PETSc.COMM_WORLD
size = comm.getSize()
rank = comm.getRank()

OptDB = PETSc.Options()
m  = 3 # OptDB.getInt('m', 8)
n  = 3 # OptDB.getInt('n', 7)

In [ ]:
''' - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
        Compute the matrix and right-hand-side vector that define
        the linear system, Ax = b.
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - '''
'''
    Create parallel matrix, specifying only its global dimensions.
    When using MatCreate(), the matrix format can be specified at
    runtime. Also, the parallel partitioning of the matrix is
    determined by PETSc at runtime.

    Performance tuning note:  For problems of substantial size,
    preallocation of matrix memory is crucial for attaining good
    performance. See the matrix chapter of the users manual for details.
'''

A = PETSc.Mat().create(comm=comm)
A.setSizes((m*n,m*n))
A.setFromOptions()
A.setPreallocationNNZ((5,5)) 

'''
    Currently, all PETSc parallel matrix formats are partitioned by
    contiguous chunks of rows across the processors.  Determine which
    rows of the matrix are locally owned.
'''
Istart,Iend = A.getOwnershipRange()

'''
    Set matrix elements for the 2-D, five-point stencil in parallel.
    - Each processor needs to insert only elements that it owns
    locally (but any non-local elements will be sent to the
    appropriate processor during matrix assembly).
    - Always specify global rows and columns of matrix entries.

    Note: this uses the less common natural ordering that orders first
    all the unknowns for x = h then for x = 2h etc; Hence you see J = Ii +- n
    instead of J = I +- m as you might expect. The more standard ordering
    would first do all variables for y = h, then y = 2h etc.
'''

for Ii in range(Istart,Iend):
    v = -1.0
    i = int(Ii/n)
    j = int(Ii - i*n)

    if (i>0):
        J = Ii - n
        A.setValues(Ii,J,v,addv=True)
    if (i<m-1):
        J = Ii + n
        A.setValues(Ii,J,v,addv=True)
    if (j>0):
        J = Ii - 1
        A.setValues(Ii,J,v,addv=True)
    if (j<n-1):
        J = Ii + 1
        A.setValues(Ii,J,v,addv=True)

    v = 4.0
    A.setValues(Ii,Ii,v,addv=True)

'''
    Assemble matrix, using the 2-step process:
    MatAssemblyBegin(), MatAssemblyEnd()
    Computations can be done while messages are in transition
    by placing code between these two statements.
'''

A.assemblyBegin(A.AssemblyType.FINAL)
A.assemblyEnd(A.AssemblyType.FINAL)
''' A is symmetric. Set symmetric flag to enable ICC/Cholesky preconditioner '''

A.setOption(A.Option.SYMMETRIC,True)

In [ ]:
'''
    Create parallel vectors.
    - We form 1 vector from scratch and then duplicate as needed.
    - When using VecCreate(), VecSetSizes and VecSetFromOptions()
    in this example, we specify only the
    vector's global dimension; the parallel partitioning is determined
    at runtime.
    - When solving a linear system, the vectors and matrices MUST
    be partitioned accordingly.  PETSc automatically generates
    appropriately partitioned matrices and vectors when MatCreate()
    and VecCreate() are used with the same communicator.
    - The user can alternatively specify the local vector and matrix
    dimensions when more sophisticated partitioning is needed
    (replacing the PETSC_DECIDE argument in the VecSetSizes() statement
    below).
'''

u = PETSc.Vec().create(comm=comm)
u.setSizes(m*n)
u.setFromOptions()

b = u.duplicate()
x = b.duplicate()

'''
    Set exact solution; then compute right-hand-side vector.
    By default we use an exact solution of a vector with all
    elements of 1.0;  
'''
u.set(1.0)
b = A(u)

'''
    View the exact solution vector if desired
'''
flg = OptDB.getBool('view_exact_sol', False)
if flg:
    u.view()

''' - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
            Create the linear solver and set various options
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - '''
ksp = PETSc.KSP().create(comm=comm)

'''
    Set operators. Here the matrix that defines the linear system
    also serves as the preconditioning matrix.
'''
ksp.setOperators(A,A)

'''
    Set linear solver defaults for this problem (optional).
    - By extracting the KSP and PC contexts from the KSP context,
    we can then directly call any KSP and PC routines to set
    various options.
    - The following two statements are optional; all of these
    parameters could alternatively be specified at runtime via
    KSPSetFromOptions().  All of these defaults can be
    overridden at runtime, as indicated below.
'''
rtol=1.e-2/((m+1)*(n+1))
ksp.setTolerances(rtol=rtol,atol=1.e-50)

'''
Set runtime options, e.g.,
    -ksp_type <type> -pc_type <type> -ksp_monitor -ksp_rtol <rtol>
These options will override those specified above as long as
KSPSetFromOptions() is called _after_ any other customization
routines.
'''
ksp.setFromOptions()

''' - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                    Solve the linear system
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - '''

ksp.solve(b,x)

''' - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
                    Check the solution and clean up
    - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - '''
diffX = x - u # x.axpy(-1.0,u)
norm = diffX.norm(PETSc.NormType.NORM_2)
its = ksp.getIterationNumber()

'''
    Print convergence information.  PetscPrintf() produces a single
    print statement from all processes that share a communicator.
    An alternative is PetscFPrintf(), which prints to a file.
'''
if norm > rtol*10:
    PETSc.Sys.Print("Norm of error {}, Iterations {}".format(norm,its),comm=comm)
else:
    if size==1:
        PETSc.Sys.Print("- Serial OK",comm=comm)
    else:
        PETSc.Sys.Print("- Parallel OK",comm=comm)

In [ ]:
x.array

In [ ]:
u.array

In [21]:
def ksvSolvePETSc(aO, bO):
    comm = PETSc.COMM_WORLD
    size = comm.getSize()
    rank = comm.getRank()
    print(size, ' ', rank)
    #OptDB = PETSc.Options()

    
    A = PETSc.Mat().create(comm=comm)
    A.setSizes([aO.shape[0],aO.shape[1]])
    A.setFromOptions()
    A.setUp()
    #A.setPreallocationNNZ((5,5)) 

    print(A.getSize())
    Istart,Iend = A.getOwnershipRange()

    for Ii in range(Istart,Iend):
        for j in range(aO.shape[1]):
            A[Ii,j] = aO[Ii,j]

    A.assemblyBegin(A.AssemblyType.FINAL)
    A.assemblyEnd(A.AssemblyType.FINAL)
    # Time parts of function individually.
    
    
    '''
    A = PETSc.Mat().create(comm=comm)
    A.setSizes([aO.shape[0],aO.shape[1]])
    A.setFromOptions()
    A.setUp()
    A.setPreallocationDense(None) 

    print(A.getSize())
    Istart,Iend = A.getOwnershipRange()

    A.setValues([list(range(aO.shape[0]))], [list(range(aO.shape[0]))], aO.flatten().tolist())

    A.assemblyBegin(A.AssemblyType.FINAL)
    A.assemblyEnd(A.AssemblyType.FINAL)
    '''
    
    '''
    A = PETSc.Mat().createDense([aO.shape[0], aO.shape[1]],comm=comm)
    A.setUp()
    A.setValues([list(range(aO.shape[0]))], [list(range(aO.shape[0]))], aO.flatten().tolist())
    A.assemblyBegin(A.AssemblyType.FINAL)
    A.assemblyEnd(A.AssemblyType.FINAL)
    '''
    #A = PETSc.Mat().createDense([aO.shape[0], aO.shape[1]],array=aO.flatten().tolist(),comm=comm)
    
    #b = PETSc.Vec().create(comm=comm)
    b = PETSc.Vec().create(comm=comm)
    b.setSizes(bO.size)
    b.setFromOptions()

    x = b.duplicate()

    for i in range(bO.size):
        b[i] = bO[i]

    #ksp = PETSc.KSP().create(comm=comm)
    ksp = PETSc.KSP().create(comm=comm)

    ksp.setOperators(A,A)


    rtol=1.e-2/(aO.size)
    ksp.setTolerances(rtol=rtol,atol=1.e-50)

    ksp.setFromOptions()
    
    before = time()
    ksp.solve(b,x)
    after = time()
    
    return [x.array, (after - before)]

In [22]:
from time import time
from scipy.linalg import lapack
import numpy as np

import sys
import petsc4py
petsc4py.init()
from petsc4py import PETSc

n = 500
A = np.random.rand(n,n)
B = np.random.rand(n,1)
A2 = np.zeros([n,n])

for I in range(n):
    for J in range(n):
        A2[I,J] = A[I,J]

before = time()
#x1 = np.linalg.solve(A,B)
_, _, x1, _ = lapack.dgesv(A2,B)
after = time()

timeLinealg = after - before

before = time()
x2, timePETSc = ksvSolvePETSc(A,B)
after = time()

timeAllPETSc = after - before

print(timeAllPETSc)

print('Norm of difference = ', np.linalg.norm(x1 - np.reshape(x2,(x2.size,1))))
print('Linalg = ', timeLinealg, ', PETSc = ', timePETSc)

1   0
(500, 500)
10.340142726898193
Norm of difference =  3.0901844552560647e-10
Linalg =  0.040839433670043945 , PETSc =  0.051215410232543945


In [ ]:
numpy.zeros([3,3])

In [ ]:
range(0,100)

In [ ]:
import petsc4py
petsc4py.init()
from petsc4py import PETSc
A = PETSc.Mat().createDense([2, 4])
A.setValue(0, 0, 3)
#A.setValues([0, 1], [2, 3], [1, 1, 1, 1]) 

In [ ]:
import petsc4py
import numpy
petsc4py.init()
from petsc4py import PETSc

In [ ]:
A = PETSc.Mat().createDense([3,3])
A.setUp()
randMatrix = numpy.random.rand(3,2)
A.setValues([list(range(3))], [0, 1], randMatrix.flatten().tolist())
A.assemblyBegin()
A.assemblyEnd()

In [ ]:
print(A.getValues(range(3), range(3)))

In [ ]:
randMatrix.flatten().tolist()

In [ ]:
randMatrix

In [ ]:
randMatrix = numpy.random.rand(3,3)
A = PETSc.Mat().createDense([3,3], array=randMatrix.flatten().tolist())
print(A.getValues(range(3), range(3)))

## SLEPc Trials
demo/ex1.py code, https://slepc.upv.es/slepc4py-current/docs/slepc4py.pdf

In [14]:
import sys, slepc4py

from petsc4py import PETSc
from slepc4py import SLEPc
import numpy

opts = PETSc.Options()
n = 30

A = PETSc.Mat(); A.create()
A.setSizes([n, n])
A.setFromOptions()
A.setUp()

rstart, rend = A.getOwnershipRange()

# first row
if rstart == 0:
  A[0, :2] = [2, -1]
  rstart += 1
# last row
if rend == n:
  A[n-1, -2:] = [-1, 2]
  rend -= 1
# other rows
for i in range(rstart, rend):
  A[i, i-1:i+2] = [-1, 2, -1]

A.assemble()

In [2]:
print(A.getValues(range(30), range(30)))

[[ 2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  2. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [18]:
E = SLEPc.EPS(); E.create()

E.setOperators(A)
E.setProblemType(SLEPc.EPS.ProblemType.HEP)
E.setDimensions(30)
E.setFromOptions()
E.solve()

#Print = PETSc.Sys.Print

In [19]:
print()
print("******************************")
print("*** SLEPc Solution Results ***")
print("******************************")
print()

its = E.getIterationNumber()
print( "Number of iterations of the method: %d" % its )

eps_type = E.getType()
print( "Solution method: %s" % eps_type )

nev, ncv, mpd = E.getDimensions()
print( "Number of requested eigenvalues: %d" % nev )

tol, maxit = E.getTolerances()
print( "Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit) )

nconv = E.getConverged()
print( "Number of converged eigenpairs %d" % nconv )

if nconv > 0:
  # Create the results vectors
  vr, vi = A.createVecs()
  #
  print()
  print("        k          ||Ax-kx||/||kx|| ")
  print("----------------- ------------------")
  for i in range(nconv):
    k = E.getEigenpair(i, vr, vi)
    error = E.computeError(i)
    if k.imag != 0.0:
      print( " %9f%+9f j %12g" % (k.real, k.imag, error) )
    else:
      print( " %12f       %12g" % (k.real, error) )
  print()


******************************
*** SLEPc Solution Results ***
******************************

Number of iterations of the method: 1
Solution method: krylovschur
Number of requested eigenvalues: 30
Stopping condition: tol=1e-08, maxit=100
Number of converged eigenpairs 30

        k          ||Ax-kx||/||kx|| 
----------------- ------------------
     3.989739        6.99152e-16
     3.959060        6.80339e-16
     3.908279        6.85878e-16
     3.837916        7.40544e-16
     3.748693        5.77119e-16
     3.641527        6.27583e-16
     3.517516        6.29939e-16
     3.377934        7.31808e-16
     3.224212        4.23001e-16
     3.057928        5.90045e-16
     2.880788        6.61057e-16
     2.694611         7.8983e-16
     2.501305        6.96956e-16
     2.302856        6.27886e-16
     2.101298        1.00615e-15
     1.898702         9.2904e-16
     1.697144        9.38761e-16
     1.498695        1.40465e-15
     1.305389        1.38283e-15
     1.119212        2.35

In [6]:
numpyA = A.getValues(range(30), range(30))

In [12]:
numpyA.ndim

2

In [17]:
numpy.linalg.eigvals(numpyA)

array([3.98973865, 3.95905988, 3.90827851, 3.83791562, 3.74869323,
       3.64152688, 3.51751625, 3.37793384, 3.22421197, 3.05792802,
       2.8807883 , 2.69461051, 2.50130506, 2.30285556, 2.10129834,
       1.89870166, 1.69714444, 1.49869494, 1.30538949, 0.01026135,
       0.04094012, 0.09172149, 0.16208438, 0.25130677, 0.35847312,
       0.48248375, 0.62206616, 0.77578803, 1.1192117 , 0.94207198])

In [ ]:
def ksvSolvePETSc(aO, bO):
    comm = PETSc.COMM_WORLD
    size = comm.getSize()
    rank = comm.getRank()
    print(size, ' ', rank)
    #OptDB = PETSc.Options()

    
    A = PETSc.Mat().create(comm=comm)
    A.setSizes([aO.shape[0],aO.shape[1]])
    A.setFromOptions()
    A.setUp()
    #A.setPreallocationNNZ((5,5)) 

    print(A.getSize())
    Istart,Iend = A.getOwnershipRange()

    for Ii in range(Istart,Iend):
        for j in range(aO.shape[1]):
            A[Ii,j] = aO[Ii,j]

    A.assemblyBegin(A.AssemblyType.FINAL)
    A.assemblyEnd(A.AssemblyType.FINAL)
    
    '''
    A = PETSc.Mat().create(comm=comm)
    A.setSizes([aO.shape[0],aO.shape[1]])
    A.setFromOptions()
    A.setUp()
    A.setPreallocationDense(None) 

    print(A.getSize())
    Istart,Iend = A.getOwnershipRange()

    A.setValues([list(range(aO.shape[0]))], [list(range(aO.shape[0]))], aO.flatten().tolist())

    A.assemblyBegin(A.AssemblyType.FINAL)
    A.assemblyEnd(A.AssemblyType.FINAL)
    '''
    
    '''
    A = PETSc.Mat().createDense([aO.shape[0], aO.shape[1]],comm=comm)
    A.setUp()
    A.setValues([list(range(aO.shape[0]))], [list(range(aO.shape[0]))], aO.flatten().tolist())
    A.assemblyBegin(A.AssemblyType.FINAL)
    A.assemblyEnd(A.AssemblyType.FINAL)
    '''
    #A = PETSc.Mat().createDense([aO.shape[0], aO.shape[1]],array=aO.flatten().tolist(),comm=comm)
    
    #b = PETSc.Vec().create(comm=comm)
    b = PETSc.Vec().create(comm=comm)
    b.setSizes(bO.size)
    b.setFromOptions()

    x = b.duplicate()

    for i in range(bO.size):
        b[i] = bO[i]

    #ksp = PETSc.KSP().create(comm=comm)
    ksp = PETSc.KSP().create(comm=comm)

    ksp.setOperators(A,A)


    rtol=1.e-2/(aO.size)
    ksp.setTolerances(rtol=rtol,atol=1.e-50)

    ksp.setFromOptions()
    
    before = time()
    ksp.solve(b,x)
    after = time()
    
    return [x.array, (after - before)]